In [57]:
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from numpy.linalg import norm
from math import log

img = mpimg.imread('nyc.jpg')
imgplot = plt.imshow(img)
# plt.show()
p1, p2 = plt.ginput(2)
plt.close("all")
print(p1, p2)
p1, p2 = np.array(p1), np.array(p2)
# p1, p2 = np.array((210, 0)), np.array((210, 730))

kernel = np.array([
    [0.125, 0.25, 0.5, 0.25, 0.125],
    [0.25, 0.5, 0.75, 0.5, 0.25],
    [0.5, 0.75, 1, 0.75, 0.5],
    [0.25, 0.5, 0.75, 0.5, 0.25],
    [0.125, 0.25, 0.5, 0.25, 0.125]
])

kx, ky = kernel.shape
imgx, imgy = img.shape[:2]

blurred = np.zeros_like(img)
margx = np.zeros((imgx, 1, 3))
ext_img = img.copy()

for i in range(kx // 2):
    ext_img = np.hstack((margx, ext_img, margx))
    
margy = np.zeros((1, imgy + kx - 1, 3))

for i in range(ky // 2):
    ext_img = np.vstack((margy, ext_img, margy))

print(img.shape)
print(ext_img.shape)
margx, margy = kx // 2 , ky // 2 

def get_temp_kernel(level):
    temp_kernel = kernel.copy()
    for i in range(level):
        min_ = np.min(kernel)
        mins = np.equal(kernel, min_)
        mins = mins.astype(int)
        mins = mins * min_
        temp_kernel += mins
    
    return temp_kernel
        

p3 = (0, 0)
temp_kernel = get_temp_kernel(int(norm(np.cross(p2-p1, p1-p3))/norm(p2-p1)))
    
for i in range(margx, imgx + margx):
    for j in range(margy, imgy + margy):
        p3 = np.array((j - margy, i - margx))
        for k in range(3):
            intensity = norm(np.cross(p2-p1, p1-p3))/norm(p2-p1)
            level = intensity / 10
            if level < 4:
                blurred[i - margx, j - margy, k] = ext_img[i-margx, j-margy, k]
                continue
            level -= 4
            if level.is_integer():
                temp_kernel = get_temp_kernel(int(level))
            blurred[i - margx, j - margy, k] = np.sum(ext_img[i-margx:i+margx+1, j-margy:j+margy+1, k] * temp_kernel) // (np.sum(temp_kernel))
     


(96.63709677419355, 380.2862903225806) (657.383064516129, 268.43145161290323)
(489, 730, 3)
(493, 734, 3)


In [58]:
plt.imshow(blurred)
plt.show()

In [61]:
sat = matplotlib.colors.rgb_to_hsv(blurred)
plt.imshow(sat)
plt.show()
sat[:,:,1] = np.minimum(2 * sat[:,:,1], np.ones_like(sat[:,:,1]))
plt.imshow(sat)
plt.show()
colored = matplotlib.colors.hsv_to_rgb(sat)
colored = colored.astype(int)
plt.imshow(colored)
plt.show()

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
